In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('front_face.csv')

In [3]:
dataset.head(5)

,id,height,weight,bmi-gt,class-gt,CJWR,WHR,PAR,ES,FWR,MEH,CFR
0,290079.jpg,1.65,97.52,35.82,d,1.19,1.64,0.02,44.50,1.38,36.34,45.37
1,290167.jpg,1.57,68.04,27.60,c,1.22,1.51,0.02,36.50,1.23,29.26,43.70
2,290173.jpg,1.63,62.14,23.39,b,1.24,1.57,0.03,32.51,1.30,24.35,35.59
3,290248.jpg,1.75,72.57,23.70,b,1.20,1.78,0.02,42.02,1.23,29.30,36.43
4,290250.jpg,1.88,94.35,26.69,c,1.23,1.66,0.02,43.00,1.28,25.92,40.13


In [4]:
len(dataset)

256

In [5]:
test_size = 0.2
test_count = int(len(dataset) * test_size)

In [6]:
raw_data = list(dataset.values)

In [7]:
test_data = list()
for _ in range(test_count):
    pick_index = np.random.choice(len(raw_data), 1, replace=False)[0]
    temp = list(raw_data.pop(pick_index))
    test_data.append(temp)

test_df = pd.DataFrame(columns=dataset.columns)
for d in test_data:
    test_df.loc[len(test_df)] = d

In [8]:
train_df = pd.DataFrame(columns=dataset.columns)
for d in raw_data:
    train_df.loc[len(train_df)] = d

In [9]:
feat_list = ['CJWR', 'WHR', 'PAR', 'ES', 'FWR', 'MEH', 'CFR']

In [10]:
x_train = train_df.loc[:, feat_list].values
y_train = train_df.loc[:, ['bmi-gt']].values

x_test = test_df.loc[:, feat_list].values
y_test = test_df.loc[:, ['bmi-gt']].values

In [11]:
# # from sklearn.preprocessing import StandardScaler
# from sklearn import preprocessing
# mm_scaler = preprocessing.MinMaxScaler()

# x_train = mm_scaler.fit_transform(x_train)
# y_train = mm_scaler.fit_transform(y_train)

# x_test = mm_scaler.fit_transform(x_test)
# y_test = mm_scaler.fit_transform(y_test)

In [11]:
from sklearn.svm import SVR

In [12]:
regressor = SVR(kernel='rbf')
regressor.fit(x_train, y_train)

C:\Users\Puff\Anaconda3\envs\ImgNrlz\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

### 使用joblib.dump匯出預測器

In [14]:
from sklearn.externals import joblib
joblib.dump(regressor,'front256_svr.pkl')

['front256_svr.pkl']

## 將剛剛輸出的模型載入

In [15]:
regressor = joblib.load('front256_svr.pkl')

### 直接預測起來

In [20]:
test_pred_y = regressor.predict(x_test)

### 使用score計算準確率

In [21]:
regressor.score(x_test, test_pred_y)

1.0

In [17]:
predict_result = list(test_pred_y)
ground_truth = list(y_test)

## 7 臉部特徵 SVR 256 張正臉 平均誤差 9%~13%

In [18]:
error = []
for pred, fact in zip(predict_result, ground_truth):
    fact = fact[0]
    dev = abs(pred - fact)
    bias = dev / fact
    error.append(bias)

print(np.mean(error))

0.1289603219184738


In [19]:
for e in error:
    print(round(e,2))

0.3
0.04
0.16
0.29
0.24
0.01
0.14
0.02
0.1
0.06
0.18
0.01
0.12
0.01
0.07
0.02
0.15
0.11
0.17
0.09
0.16
0.02
0.3
0.04
0.18
0.2
0.09
0.0
0.13
0.25
0.11
0.22
0.11
0.24
0.06
0.05
0.2
0.1
0.18
0.11
0.05
0.06
0.11
0.11
0.04
0.23
0.13
0.05
0.26
0.21
0.3


In [19]:
import pickle

In [20]:
s = pickle.dumps(regressor)

In [22]:
# Save the model to a file
f = open('BMI_SVR.model', 'wb')
f.write(s)
f.close()

In [24]:
from sklearn.metrics import classification_report

In [26]:
print(classification_report(y_test, test_pred_y))

ValueError: continuous is not supported